# Данные

In [1]:
import pandas as pd
import gzip
import json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Appliances.json.gz')
df_meta = getDF('meta_Appliances.json.gz')

In [2]:
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5.0,2,False,"11 27, 2013",A3NHUQ33CFH3VM,1118461304,{'Format:': ' Hardcover'},Greeny,Not one thing in this book seemed an obvious o...,Clear on what leads to innovation,1385510400,NaN
1,5.0,NaN,False,"11 1, 2013",A3SK6VNBQDNBJE,1118461304,{'Format:': ' Kindle Edition'},Leif C. Ulstrup,I have enjoyed Dr. Alan Gregerman's weekly blo...,Becoming more innovative by opening yourself t...,1383264000,NaN
2,5.0,NaN,False,"10 10, 2013",A3SOFHUR27FO3K,1118461304,{'Format:': ' Hardcover'},Harry Gilbert Miller III,Alan Gregerman believes that innovation comes ...,The World from Different Perspectives,1381363200,NaN
3,5.0,NaN,False,"10 9, 2013",A1HOG1PYCAE157,1118461304,{'Format:': ' Hardcover'},Rebecca Ripley,"Alan Gregerman is a smart, funny, entertaining...",Strangers are Your New Best Friends,1381276800,NaN
4,5.0,10,False,"09 7, 2013",A26JGAM6GZMM4V,1118461304,{'Format:': ' Hardcover'},Robert Morris,"As I began to read this book, I was again remi...","How and why it is imperative to engage, learn ...",1378512000,NaN


In [3]:
df_meta.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,"[Appliances, Refrigerators, Freezers & Ice Mak...","class=""a-keyvalue prodDetTable"" role=""present...",[],,Tupperware Freezer Square Round Container Set ...,[],,Tupperware,[Each 3-pc. set includes two 7/8-cup/200 mL an...,"[>#39,745 in Appliances (See top 100)]",[],{},Appliances,,"November 19, 2008",,7301113188,[],[]
1,"[Appliances, Refrigerators, Freezers & Ice Mak...","class=""a-keyvalue prodDetTable"" role=""present...",[2 X Tupperware Pure & Fresh Unique Covered Co...,,2 X Tupperware Pure &amp; Fresh Unique Covered...,[],,Tupperware,[2 X Tupperware Pure & Fresh Unique Covered Co...,"[>#6,118 in Appliances (See top 100)]",[B004RUGHJW],{},Appliances,,"June 5, 2016",$3.62,7861850250,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,"[Appliances, Parts &amp; Accessories]",,[],,The Cigar - Moments of Pleasure,[],,The Cigar Book,[],"[>#1,861,816 in Home &amp; Kitchen (See Top 10...","[B01HCAVSLK, 1632206579]",{},Amazon Home,,,$150.26,8792559360,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,"[Appliances, Parts & Accessories]","class=""a-keyvalue prodDetTable"" role=""present...","[Multi purpost descaler, especially suited to ...",,Caraselle 2X 50G Appliance Descalene,[],,Caraselle,[],"[>#1,654,505 in Tools & Home Improvement (See ...",[],{},Tools & Home Improvement,,"December 17, 2014",.a-box-inner{background-color:#fff}#alohaBuyBo...,9792954481,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,"[Appliances, Parts & Accessories, Range Parts ...","class=""a-keyvalue prodDetTable"" role=""present...",[Full gauge and size beveled-edge; furnished w...,,EATON Wiring 39CH-SP-L Arrow Hart 1-Gang Chrom...,[],,EATON Wiring,[Returns will not be honored on this closeout ...,"[>#3,066,990 in Tools & Home Improvement (See ...",[],{},Tools & Home Improvement,,"January 16, 2007",$3.43,B00002N5EL,[],[]


# Поиск упоминаний

## Словарный метод

Словарь NE можно подготовить с помощью данных файла meta_Appliances. Например, из колонки "title" можно с помощью Spacy достать возможные NP. Например: range cord, humidifier, humidifier filter pad. Однако после потребуется доработка вручную
- Недостатки:
    * Слишком трудозатратный и неточный процесс. На составление и редактирование словаря длдя большого количества данных уйдет много времени
    * Неоднозначность. Если составлением словаря занимаются несколько человек, то их разметка может отличаться

In [4]:
#!python -m spacy download en_core_web_sm
import spacy
import re

df_meta['title'] = [a.lower().replace(b.lower(), '').strip() for a, b in zip(df_meta['title'], df_meta['brand'])]

nlp = spacy.load("en_core_web_sm")
NE = []
for text in df_meta['title']:
    new_text = ''
    for word in text.split():
        if re.fullmatch(r'[A-Za-z]{2,}', word):
            new_text += word + ' '
    doc = nlp(new_text)
    for chunk in doc.noun_chunks:
        NE.append(chunk)
        
NE_final = [] 
for i in NE: 
    if str(i) not in NE_final: 
        NE_final.append(str(i)) 
NE_final[:20]

['freezer square round container set',
 'pure fresh unique covered cool cubes ice tray',
 'purple',
 'lid',
 'cubes',
 'the cigar moments',
 'pleasure',
 'appliance descalene',
 'arrow hart chrome power outlet',
 'electrical pack',
 'nema',
 'mounting straight industrial side steel black',
 'flush mount receptacle',
 'range cord',
 'dryer cord',
 'appliance and power tool extension',
 'dryer outlet receptacle',
 'industrial grade straight blade receptacle amp volt nema black decora',
 'aluminum dryer vent kit',
 'white']

## Rule-based approach Spacy

NE часто могут встречаться в подобных контекстах:
 - "I had this xxx"
 - "great xxx"
 - "purchase xxx"
 - "had xxx installed"
 - "this is a (high quality) xxx"
Их можно попробовать выделить с помощью Spacy matcher (https://spacy.io/usage/rule-based-matching)

In [ ]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)
nlp = spacy.load("en_core_web_sm")

pattern = [{'ORTH': 'this'}, {'POS': 'NOUN'}, {'LEMMA': 'work'}, {'LEMMA': 'perfect'}]
matcher.add('pattern', [pattern])
for text in df['reviewText']:
    if type(text) == str:
        doc = nlp(text)
        matches = matcher(doc)
        for match_id, start, end in matches:
            string_id = nlp.vocab.strings[match_id]  # Get string representation
            span = doc[start:end]  # The matched span
            print(span.text)

this item worked perfect
this knob worked perfect
this item work perfect
this fuse worked perfect
this style works perfect
this product work perfect
this part works perfect
this one worked perfect


### Недостатки
- Подходит только для текстов одной тематики/формата. С помощью этого способа довольно сложно выделить NE в отзывах из-за их неоднородности.

### Преимущества
- Мало неоднородности в отличие от словарного подхода

## CRF

Задачу можно также решить с помощью Conditional random field, но для этого понадобится подготовленный размеченный датасет для обучения модели

### Недостатки:
- Только forward firection

### Достоинства
- Достаточно хороший результат на новых данных